In [1]:
import w2vAida
import analogy_completion

Using gpu device 0: GeForce GTX 960M (CNMeM is disabled, cuDNN not available)


In [2]:
model_file = "D:/data/analogy/pratima_w2v_models/aida.model.bin"
#model_file = "D:/data/analogy/pratima_w2v_models/s2v.model.bin"
vocab_file = "D:/data/analogy/pratima_w2v_models/aida.vocab"
golddata_file = "../testData/AGS/AGS-V02.txt"
output_file = "./result/test.txt"

In [3]:
def loadGoldData(dataset):
    with open(dataset, 'r') as file:
        lines = file.readlines()
        simGold = []
        i = 0
        while i < len(lines):
            line = lines[i].strip()
            i += 1
            if line.startswith(":"):
                continue
            if line.startswith("\n"):
                i += 1
                continue
            splits = line.split(" ")
            correctedsplits = [ ]
            correctedsplits.append(splits[0])
            correctedsplits.append(splits[1])
            correctedsplits.append(splits[2])
            correctedsplits.append(splits[3])
            correctedsplits.append(splits[4])
            correctedsplits.append(splits[5])
            correctedsplits.append(splits[6])
            correctedsplits.append(splits[7])
            #correctedsplits.append(splits[8])
            simGold.append(correctedsplits)
            #print(correctedsplits)
        return simGold

In [4]:
analogydataset = loadGoldData(golddata_file)

In [5]:
model=w2vAida.Word2Vec.load_word2vec_format(fname=model_file, binary=True, fvocab=vocab_file, delimeter=":")

load local with delimeter :


In [6]:
model.n_similarity(['sushi|noun', 'shop|noun'], ['japanese|noun', 'restaurant|noun'])

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [ ]:
## completion query!
for value in analogydataset:
    temp = model.most_similar_new(positive=[value[2], value[1]], negative=[value[0]],topn=1)
    for values in temp:
        if ':' in values[0]:
            v = values[0]
            v1 = v.split(':')
            similarityvalue=v1[0]
        else:
            similarityvalue=values[0]
    print("{}:{} :: {}:{}  >> {}".format(value[0], value[1], value[2], value[3], similarityvalue))

In [9]:
duplicatecount=0.0
count=0.0
countlines=0.0
previouscheckvalue=0.0
total=0.0
nextval = 0

for value in analogydataset:
    #if(value[6]==word):
    r_similarity = model.n_similarity_new([value[0], value[1]],[value[2],value[3]])
    print("{}:{} :: {}:{}  >> {}".format(value[0], value[1], value[2], value[3], r_similarity))
    
    similarityrating=value[5]
    countlines+=1
    originalRating=value[4]
    originalcheckvalue=value[0]
    if (originalRating >= '4.5' and similarityrating >= '0.70'):
        count+=1
        if(previouscheckvalue==originalcheckvalue):
            count=duplicatecount
            duplicatecount=count
            previouscheckvalue=originalcheckvalue
        #print(originalcheckvalue)
        if nextval!=originalcheckvalue:
            #print(nextval,originalcheckvalue)
            total+=1
        nextval = originalcheckvalue

apple:apple_inc steve_jobs:steve_jobs microsoft bill_gates:bill_gates
apple:steve_jobs :: microsoft:bill_gates  >> 0.7709794995660414
tired sleep thirsty drink
tired:sleep :: thirsty:drink  >> 0.6904755623185721
tired sleep sad:sad_sam_jones cry:cry_
tired:sleep :: sad:cry  >> 0.6949892859740119
tired sleep hungry eat
tired:sleep :: hungry:eat  >> 0.7616618911790528
tired sleep fresh:doug_e work:the_adventures_of_tom_sawyer
tired:sleep :: fresh:work  >> 0.6610758869281049
tired sleep lazy:x couch:tim_couch
tired:sleep :: lazy:couch  >> 0.7329755054529271
tired sleep bored:tunnel_boring_machine read:reading_
tired:sleep :: bored:read  >> 0.65876378634391
tired sleep happy:happy_mondays sit
tired:sleep :: happy:sit  >> 0.7570925285340628
lifejacket boat:the_boat_race belt car:central_african_republic
lifejacket:boat :: belt:car  >> 0.7343746803796274
lifejacket boat:the_boat_race seatbelt airplane:airplane
lifejacket:boat :: seatbelt:airplane  >> 0.8338263151809748
lifejacket boat:the_bo

KeyboardInterrupt: 

In [ ]:
print(model.vocab["gemap"])

In [ ]:
len(model.index2word)

In [8]:
for word,value in model.vocab.items():
    if value.count and value.count<1000:
        print(word+" "+str(value.count))

cecil_heftel:cecil_heftel 10
orduspor 5
chessboard 250
kano_state:kano_state 12
shrewton 8
higginsville:higginsville 12
fhimah:lamin_khalifah_fhimah 19
priozersky 5
delta_north:delta_north 5
dwight_yorke:dwight_yorke 53
uster 41
wtvc:wtvc 14
ray_greene:ray_greene_ 7
marine_city:marine_city 8
_beito:david_t 24
hkadb 9
chilkoot 52
bostonroads 20
gnessin 13
appledouble 10
oak_st 5
jinian:jinian_wilde 6
babbello 6
spier 72
rosenior 5
s_zaman 81
bocksdorf 9
reconfiguration 247
bukhsh 5
james_higgins:james_h 6
mayo_bridge:mayo_bridge 5
leighlin:old_leighlin 26
capd 5
cito_gaston:cito_gaston 14
kurogasa 21
mosey 46
rhoon 7
korobeiniki:korobeiniki 8
eraniel 6
motl:lubo 12
cross_road:cross_road 13
bhne 34
herbois 21
litsea:litsea 8
grimmelshausen:hans_jakob_christoffel_von_grimmelshausen 10
agano:agano_river 13
anna_dei_lombardi 10
hagelstein 7
gaelic_games:gaelic_athletic_association 89
red_miller:red_miller 5
tokus 24
thuille 8
evaluative 105
vitoldo 5
percy_mayfield:percy_mayfield 14
paraphy

ValueError: I/O operation on closed file